# Visualization n° 1

## Loading the data

In [158]:
import altair as alt
import os
import pandas as pd
import geopandas as gpd # Requires geopandas -- e.g.: conda install -c conda-forge geopandas
alt.data_transformers.enable('json') # Let Altair/Vega-Lite work with large data sets

pass

In [159]:
names = pd.read_csv("dpt2020.csv", sep=";")
names.drop(names[names.preusuel == '_PRENOMS_RARES'].index, inplace=True)
names.drop(names[names.dpt == 'XX'].index, inplace=True)

names.sample(5)

,sexe,preusuel,annais,dpt,nombre
1216658,1,MILHAN,2020,31,6
1605653,1,TOM,1990,75,37
1087386,1,MARC,1946,39,8
2729722,2,JULIETTE,1933,73,9
984788,1,LÉON,1907,06,21


In [177]:
depts = gpd.read_file('departements-version-simplifiee.geojson')

## Data processing and Visualization

In [161]:
grouped = names.groupby(['annais', 'preusuel', 'sexe'], as_index=False).sum()
grouped

,annais,preusuel,sexe,nombre
0,1900,ABEL,1,382
1,1900,ABRAHAM,1,9
2,1900,ACHILLE,1,152
3,1900,ACHILLES,1,4
4,1900,ADAM,1,9
...,...,...,...,...
257341,2020,ÉVA,2,156
257342,2020,ÉVAN,1,62
257343,2020,ÉZIO,1,12
257344,2020,ÉZÉCHIEL,1,11


In [270]:
top_5_names = grouped.groupby('preusuel')["nombre"].sum().sort_values()[::-1][:5].index.tolist()
top_6_10_names = grouped.groupby('preusuel')["nombre"].sum().sort_values()[::-1][5:10].index.tolist()
print("Top 5 names from 1900 to 2019 : \n{}".format(top_5_names))
print("Top 6-10 names from 1900 to 2019 : \n{}".format(top_6_10_names))

Top 5 names from 1900 to 2019 : 
['MARIE', 'JEAN', 'PIERRE', 'MICHEL', 'ANDRÉ']
Top 6-10 names from 1900 to 2019 : 
['JEANNE', 'PHILIPPE', 'LOUIS', 'RENÉ', 'ALAIN']


In [271]:
subset1 = grouped.loc[grouped["preusuel"].apply(lambda x : any(x == name for name in top_5_names))]
subset2 = grouped.loc[grouped["preusuel"].apply(lambda x : any(x == name for name in top_6_10_names))]

In [289]:
base1 = alt.Chart(subset1).mark_area(opacity=0.3).encode(
    x=alt.X("annais:T", title='Année'),
    y=alt.Y("nombre:Q", stack=None, title='Nombre de fois que le prénom a été donné'), 
    color=alt.Color("preusuel:N", title = "Top 10 des prénoms")
    ).properties(width=800, height=400
    )

base2 = alt.Chart(subset2).mark_area(opacity=0.3).encode(
    x=alt.X("annais:T", title='Année'),
    y=alt.Y("nombre:Q", stack=None, title='Nombre de fois que le prénom a été donné'), 
    color=alt.Color("preusuel:N")
    ).properties(width=800, height=400
    )

In [290]:
base1 & base2

alt.VConcatChart(...)